In [282]:
import pandas as pd
import os
import numpy as np
import sys
sys.path.append('..')

In [283]:
data = pd.read_csv('../data/interim/all_train_data.csv')

In [284]:
data.columns

Index(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
     

In [285]:
pct = np.percentile(data.loc[:, 'area_percentage'].fillna(np.mean(data.loc[:, 'area_percentage'])), 95)
print(pct)
print(data.shape)
data = data.loc[data.loc[:, 'area_percentage'] < pct]
print(data.shape)

16.0
(260601, 39)
(247044, 39)


In [286]:
from sklearn.preprocessing import FunctionTransformer
import numpy as np
#height_percentage
# Function to create interaction terms
def create_interaction_terms(X):
    # Assuming X is a dataframe with the columns geo_level_1_id, geo_level_2_id, geo_level_3_id
    X_new = X.copy()
    #X_new['areaHeight'] = X_new['height_percentage'] / X_new['area_percentage']
    X_new['geo1times2'] = X_new['geo_level_1_id'] * X_new['geo_level_2_id']
    X_new['geo1times3'] = X_new['geo_level_1_id'] * X_new['geo_level_3_id']
    X_new['geo2times3'] = X_new['geo_level_2_id'] * X_new['geo_level_3_id']
    X_new['geoall'] = X_new['geo_level_2_id'] * X_new['geo_level_3_id'] * X_new['geo_level_1_id']
    return X_new[['geo1times2', 'geo1times3', 'geo2times3', 'geoall']]#, 'areaHeight']]  # Return the interaction terms only

# Creating the FunctionTransformer
interaction_transformer = FunctionTransformer(create_interaction_terms)


In [287]:
numerical_df = data.select_dtypes(exclude=['object'])
numerical_df.describe()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
count,247044.000000,247044.000000,247044.000000,247044.000000,247044.000000,247044.000000,247044.000000,247044.000000,247044.000000,247044.000000,...,247044.000000,247044.000000,247044.000000,247044.000000,247044.000000,247044.000000,247044.000000,247044.000000,247044.000000,247044.000000
mean,13.784293,705.920686,6263.536168,2.122193,26.682757,7.308403,5.380519,0.088102,0.782917,0.034581,...,0.027894,0.005963,0.000591,0.000202,0.000935,0.000146,0.000113,0.000085,0.005036,2.254121
std,7.968545,410.536461,3642.368254,0.711754,73.527319,2.774035,1.842914,0.283443,0.412260,0.182716,...,0.164669,0.076987,0.024303,0.014225,0.030564,0.012071,0.010646,0.009219,0.070783,0.603290
min,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,7.000000,356.000000,3086.750000,2.000000,10.000000,5.000000,4.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,12.000000,710.000000,6277.000000,2.000000,15.000000,7.000000,5.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,21.000000,1051.000000,9414.000000,2.000000,30.000000,9.000000,6.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,30.000000,1427.000000,12567.000000,9.000000,995.000000,15.000000,32.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000


In [288]:
from scipy.stats import skew

def get_right_skewed_columns(df, skew_threshold=0.5):
    """
    Returns the names of columns that are right-skewed based on the skewness value, excluding binary columns.
    
    Parameters:
    - df: The input DataFrame (numerical columns only).
    - skew_threshold: The skewness threshold above which a column is considered right-skewed (default is 0.5).
    
    Returns:
    - List of column names that are right-skewed.
    """
    right_skewed_columns = []
    
    # Iterate through each column in the dataframe
    for col in df.columns:
        # Check if the column has more than 2 unique values (to avoid binary columns)
        if df[col].nunique() > 2:
            # Calculate skewness for each column
            col_skewness = skew(df[col].dropna())  # Drop NaN values to avoid issues
            
            # Check if the skewness is above the specified threshold (indicating right-skewness)
            if col_skewness > skew_threshold:
                right_skewed_columns.append(col)
    
    return right_skewed_columns


In [289]:
# # Select numerical columns
# numerical_df = data.select_dtypes(exclude=['object'])

# # Get the right-skewed columns
right_skewed_cols = get_right_skewed_columns(numerical_df)

print("Right-skewed columns:", right_skewed_cols)


Right-skewed columns: ['count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage', 'count_families']


In [290]:
# import numpy as np

# def apply_log_transformation(df, columns):
#     """
#     Applies log transformation to the specified columns of the DataFrame.
    
#     Parameters:
#     - df: The input DataFrame.
#     - columns: List of column names to apply the log transformation on.
    
#     Returns:
#     - DataFrame with log-transformed columns.
#     """
    
#     df_transformed = df.copy()
    
#     # Apply log transformation to each specified column
#     for col in columns:
#         # Add a small constant to avoid log(0) and handle any zeros or negatives
#         df_transformed[col] = np.log1p([col])
    
#     return df_transformed


In [291]:
# Get the right-skewed columns (excluding binary columns)
#right_skewed_cols = get_right_skewed_columns(numerical_df)
#numerical_df = apply_log_transformation(numerical_df, right_skewed_cols)
#data[right_skewed_cols] = numerical_df[right_skewed_cols]

In [292]:
#pct = np.percentile(data.loc[:, 'age'].fillna(np.mean(data.loc[:, 'age'])), 99)
#print(pct)
#print(data.shape)
#data = data.loc[data.loc[:, 'age'] < pct]
#print(data.shape)

In [293]:
def log_transform(X):
    # Apply log1p transformation (log(1 + x)) to avoid issues with zero values
    return np.log1p(X)

# Create a FunctionTransformer for log transformation
log_transformer = FunctionTransformer(log_transform)


In [294]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# Custom transformer for the age-based transformation
class AgeTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, age_column='age'):
        self.age_column = age_column
        self.percentile_ = None

    def fit(self, X, y=None):
        # Calculate the 99th percentile of the 'age' column and store it
        self.percentile_ = np.percentile(X[self.age_column].fillna(np.mean(X[self.age_column])), 99)
        return self

    def transform(self, X):
        X_copy = X.copy()
        
        # Add a new 'old' column to indicate if the age exceeds the 99th percentile
        X_copy['old'] = np.where(X_copy[self.age_column] >= self.percentile_, 1, 0)
        
        # Cap the age to 100 where the 'old' column is 1
        X_copy.loc[X_copy['old'] == 1, self.age_column] = 100
        
        return X_copy
    



In [295]:

class geoTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, cols, min):
        self.cols = cols
        self.min = min

    def fit(self, X, y=None):
        self.value_counts_ = {}
        for col in self.cols:
            self.value_counts_[col] = X[col].value_counts()
        return self

    def transform(self, X):
        X_transformed = X.copy()
        for col in self.cols:
            counts = self.value_counts_[col]
            mask = X[col].map(counts) < self.min
            X_transformed.loc[mask, col] = -1
        return X_transformed



In [296]:
x = data.drop(columns=['damage_grade'])
y = data.damage_grade

In [297]:
y = y.replace({1: 0, 2: 1, 3: 2})

In [298]:
y

0         2
1         1
2         2
3         1
4         2
         ..
260596    1
260597    2
260598    2
260599    1
260600    2
Name: damage_grade, Length: 247044, dtype: int64

In [299]:
y.value_counts()

damage_grade
1    141177
2     84323
0     21544
Name: count, dtype: int64

In [300]:
#x.old.value_counts()

In [301]:
#numerical_df = data.select_dtypes(exclude=['object'])
#categorical_df = data.select_dtypes(include=['object'])

In [302]:
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in x.columns if  x[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in x.columns if x[cname].dtype in ['int32', 'int64', 'float64']]

In [303]:
categorical_cols

['land_surface_condition',
 'foundation_type',
 'roof_type',
 'ground_floor_type',
 'other_floor_type',
 'position',
 'plan_configuration',
 'legal_ownership_status']

In [304]:
#plt.hist(numerical_df.age)

In [305]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,StandardScaler, MinMaxScaler

#train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42, shuffle=True)

In [306]:
y_train.value_counts()

damage_grade
1    98897
2    58949
0    15084
Name: count, dtype: int64

In [307]:
y_test.value_counts()

damage_grade
1    42280
2    25374
0     6460
Name: count, dtype: int64

In [308]:
from category_encoders import BaseNEncoder

In [309]:
#from src.eda_first import summarize_dataframe
#summarize_dataframe(X_train)

In [310]:
#create numerical transformer

numerical_transformer = Pipeline([('imputer', SimpleImputer(strategy='mean'))])

#create categorical transformer
#categorical_transformer = Pipeline(steps=[ ('imputer', SimpleImputer(strategy='most_frequent')),
#                                            ('onehot', OneHotEncoder(handle_unknown='ignore'))
#                                            ])

base_encoder_columns = ['land_surface_condition','geo_level_1_id', 'geo_level_2_id','geo_level_3_id',
 'foundation_type',
 'roof_type',
 'ground_floor_type',
 'other_floor_type',
 'position',
 'plan_configuration',
 'legal_ownership_status']

base_encoder = Pipeline(steps=[
    ('base_encoder', BaseNEncoder(cols=base_encoder_columns, base=3))
])
geo_columns = ['geo_level_1_id', 'geo_level_2_id','geo_level_3_id']
geo_encoder = Pipeline(steps=[
    ('geo_encoder', geoTransformer(cols=geo_columns, min=5))
])

age_transformer = Pipeline(steps=[
    ('age_transform', AgeTransformer(age_column='age'))  # Apply age transformation
])

In [311]:
# Combine the transformations using ColumnTransformer
#preprocessor = ColumnTransformer(transformers=[
#    ('base_name', base_encoder, base_encoder_columns)])  # TargetEncoder for 'town'
#    ('num', numerical_transformer, numerical_cols)])

In [312]:
# Updated ColumnTransformer with the log transformer
preprocessor = ColumnTransformer(transformers=[
    #('geo_encoder', geo_encoder, geo_columns),
    ('base_name', base_encoder, base_encoder_columns),  # BaseNEncoder for categorical columns
    ('age_transform', age_transformer, ['age']),  # Custom transformer for the 'age' column
    ('num', 'passthrough', numerical_cols),  # Pass numerical columns through without transformation
    ('interaction', interaction_transformer, ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'height_percentage', 'area_percentage']),  # Interaction terms
    ('log_transform', log_transformer, right_skewed_cols)  # Apply log transformation to specified columns
])

In [313]:
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
# XGBoost
xgb = XGBClassifier(
    n_estimators=600,
    learning_rate=0.2669112505018992,
    max_depth=5,
    random_state=42,
    reg_lambda=1.2259716591605452,
    subsample=0.704976942819638,
    colsample_bytree=0.9,
    min_child_weight=4,
    alpha= 0.14170716330946964,    # Added L1 regularization
    eval_metric='mlogloss',  # Consider custom loss for ordinal
    objective='multi:softmax',  # Using softmax but can tweak for ordinal
    num_class=3  # Assuming 3 ordinal classes
)

rf_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Step 1: Preprocessing
    ('xgboost', xgb)  # Step 3: Model training
])

# Preprocessing of training data, fit model 
#rf_pipe.fit(X_train, y_train)

# Preprocessing of training data, fit model after upsampling!
rf_pipe.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
rf_preds = rf_pipe.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, rf_preds)
print('Accuracy for XGBoost:', accuracy)

# Detailed classification report
print('Classification Report:\n', classification_report(y_test, rf_preds))

Accuracy for XGBoost: 0.7457295517715951
Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.49      0.57      6460
           1       0.75      0.85      0.79     42280
           2       0.76      0.64      0.70     25374

    accuracy                           0.75     74114
   macro avg       0.73      0.66      0.69     74114
weighted avg       0.74      0.75      0.74     74114



In [314]:
# Preprocessing of validation data, get predictions
rf_preds = rf_pipe.predict(X_train)

# Evaluate the model
accuracy = accuracy_score(y_train, rf_preds)
print('Accuracy for XGBoost:', accuracy)

# Detailed classification report
print('Classification Report:\n', classification_report(y_train, rf_preds))

Accuracy for XGBoost: 0.7980743653501416
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.63      0.71     15084
           1       0.79      0.89      0.84     98897
           2       0.81      0.69      0.75     58949

    accuracy                           0.80    172930
   macro avg       0.80      0.74      0.76    172930
weighted avg       0.80      0.80      0.79    172930



In [315]:
test_data = pd.read_csv('../data/raw/test_values.csv')

In [316]:
X_test_final = test_data.building_id

In [317]:
X_test_final

0         300051
1          99355
2         890251
3         745817
4         421793
          ...   
86863     310028
86864     663567
86865    1049160
86866     442785
86867     501372
Name: building_id, Length: 86868, dtype: int64

In [318]:
rf_preds = rf_pipe.predict(test_data)
rf_preds = pd.Series(rf_preds)
rf_preds 

0        2
1        1
2        1
3        0
4        2
        ..
86863    1
86864    2
86865    1
86866    1
86867    0
Length: 86868, dtype: int32

In [319]:
df_concatenated = pd.concat([X_test_final, rf_preds], axis=1)
df_concatenated
df_concatenated = df_concatenated.rename(columns={0: 'damage_grade'})
df_concatenated
df_concatenated['damage_grade'] = df_concatenated['damage_grade'].replace({0: 1, 1: 2, 2: 3})
df_concatenated.to_csv('../data/processed/ATsubmission1.csv', index=False)